In [1]:
import numpy as np
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC


In [2]:
driver = webdriver.Chrome()

In [3]:
def extract_text(url,part_no):
    
    driver.get(url)
    time.sleep(15)
    
    
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    #part_no = soup.find('span',attrs={'class':'part-info-spec__value'})
    #try:
    #    part_no=part_no.text[1:-1]
    #except:
    #    driver.get(url)
    #    time.sleep(10)
    #    content = driver.page_source
    #    soup = BeautifulSoup(content)
   
    #    part_no = soup.find('span',attrs={'class':'part-info-spec__value'})
    #    part_no=part_no.text[1:-1]
    
    part_no = part_no
    sp=part_no.split("-")
    if len(sp)==1:
        part_no=str(part_no[:5])+"-"+part_no[5:]
    
    brand=soup.find('a',href=True, attrs={'class':'part-info-top__brand'})
    try:
        if brand.text:
            brand=brand.text[1:-1]
        else:
            brand=brand.get("href").split("-")[-1][:-1]
    except:
        print("error in brand")
    
    #brand= "Maruti Suzuki"
    
    desc=soup.find('h2', attrs={'class':'part-info-heading'})
    desc=desc.text[1:-1]

    
    lol=[]
    for a in soup.findAll('a',href=True, attrs={'class':'ng-star-inserted'}):
        lol.append(a)
    try:
        category = lol[3].text[1:-1]
    except:
        category = None
    try:
        sub_category=lol[4].text[1:-1]
    except:
        sub_category=None
    
    price=soup.find('span', attrs={'class':'part-info-price__mrp'})
    try:
        price=price.text[6:]
        price=price.split(",")
        pric=""
        for i in price:
            pric=pric+i
        price=int(pric)
    except:
        price = None
        
    
    
    com_len=[]
    for a in soup.findAll('li', attrs={'class':'compatibility-nav__list__item'}):
        com_len.append(a.text[1:-1])
        
    if len(com_len)==0:
        print("no data")
        return

    df=[]
    for i in range(len(com_len)):
        make_name_li = com_len[i].split(" ")[:-1]
        make_name=" ".join(make_name_li)
        make_len = int(com_len[i].split(" ")[-1][1:-1])
        
        if i>4 and i!=8 and i!=11 and i!=14 and i!=17 and i!=20:
            driver.find_element(By.XPATH, '//button[@class="compatibility-nav__arrow compatibility-nav__arrow--next ng-star-inserted"]').click()
            time.sleep(1)
        
        button=WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, f"//li[contains(text(),'{com_len[i]}')]")))
        button.click()
        time.sleep(2)
        
        
        if make_len>10:
            WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "span.btn.btn-sm"))).click()
        content = driver.page_source
        soup = BeautifulSoup(content)
        time.sleep(2)
        

        li_model=[]
        for a in soup.findAll('div', attrs={"data-head-title":"Model"}):
            li_model.append(a.text[1:-1])

        li_year=[]
        for a in soup.findAll('div', attrs={"data-head-title":"Year"}):
            li_year.append(a.text[1:-1])

        li_engine=[]
        for a in soup.findAll('div', attrs={"data-head-title":"Engine"}):
            li_engine.append(a.text[1:-1])

        li_power=[]
        for a in soup.findAll('div', attrs={"data-head-title":"Power (hp)"}):
            li_power.append(a.text[1:-1])

        li_fuel=[]
        for a in soup.findAll('div', attrs={"data-head-title":"Fuel type"}):
            li_fuel.append(a.text[1:-1])

        li_engine_type=[]
        for a in soup.findAll('div', attrs={"data-head-title":"Engine type"}):
            li_engine_type.append(a.text[1:-1])

        
        for i in range(len(li_model)):
            df.append([part_no,brand,"",desc,price,category,sub_category,"",make_name,li_model[i],li_year[i],li_engine[i],li_power[i],li_fuel[i],li_engine_type[i]])

        
    return df

In [4]:
lol=["""177.935-161"""]

skus=lol[0].split()
len(skus)

1

In [5]:
def extract_sku(url):
    li=[]
    for i in url:
        li.append(["https://boodmo.com/search/"+i,i])
    return li
uri=extract_sku(skus)
len(uri)

1

In [6]:
def extract_url(url):
    
    driver.get(url)
    time.sleep(10)
    
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    li=[]
    a=soup.findAll('a',href=True,attrs={'class':'product-item-list__images'})
    try:
        a=a[0].get("href")
        li.append("https://boodmo.com"+a)
    except:
        print("no url")
    return li

In [7]:
li=[]

In [8]:
for i in range(len(uri)):
    a=extract_url(uri[i][0])
    for j in a:
        li.append([j,uri[i][1]])
    print(i)

0


In [9]:
len(li)

1

In [10]:
df1 = pd.DataFrame()
df1

""


In [11]:
for i in range(len(li)):
    l=extract_text(li[i][0],li[i][1])
    df2=pd.DataFrame(l)
    df1=pd.concat([df1, df2])
    print(i)

0


In [13]:
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,177.935-161,hella,,"Wiper Blade 16""",179,Windscreen Cleaning System,Wiper Blade,,MARUTI,ALTO 1.1L VX (TYPE 1),07.2000 - 01.2005,1.1 L,62 h.p.,Petrol,F10D
1,177.935-161,hella,,"Wiper Blade 16""",179,Windscreen Cleaning System,Wiper Blade,,MARUTI,ALTO 0.8L LX (TYPE 1),07.2000 - 05.2005,0.8 L,47 h.p.,Petrol,F8D
2,177.935-161,hella,,"Wiper Blade 16""",179,Windscreen Cleaning System,Wiper Blade,,MARUTI,ALTO 0.8L LXI (TYPE 1),07.2000 - 05.2005,0.8 L,47 h.p.,Petrol,F8D
3,177.935-161,hella,,"Wiper Blade 16""",179,Windscreen Cleaning System,Wiper Blade,,MARUTI,ALTO 0.8L VXI (TYPE 1),07.2000 - 05.2005,0.8 L,47 h.p.,Petrol,F8D
4,177.935-161,hella,,"Wiper Blade 16""",179,Windscreen Cleaning System,Wiper Blade,,MARUTI,ALTO 1.1L VXI (TYPE 1),07.2000 - 01.2005,1.1 L,62 h.p.,Petrol,F10D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161,177.935-161,hella,,"Wiper Blade 16""",179,Windscreen Cleaning System,Wiper Blade,,CITROEN,C5 AIRCROSS SUV 2.0L FEEL,04.2021 - 02.2022,2 L,174 h.p.,Diesel,DW10FC
1162,177.935-161,hella,,"Wiper Blade 16""",179,Windscreen Cleaning System,Wiper Blade,,CITROEN,C5 AIRCROSS SUV 2.0L SHINE,04.2021 - 02.2022,2 L,174 h.p.,Diesel,DW10FC
1163,177.935-161,hella,,"Wiper Blade 16""",179,Windscreen Cleaning System,Wiper Blade,,CITROEN,C5 AIRCROSS SUV F/L 2.0L SHINE,09.2022 - now,2 L,174 h.p.,Diesel,DW10FC
1164,177.935-161,hella,,"Wiper Blade 16""",179,Windscreen Cleaning System,Wiper Blade,,CITROEN,E-C3 29kW LIVE,01.2023 - now,N/A,56 h.p.,Electric,SYNCHRONOUS MOTOR


In [14]:
len(pd.unique(df1[0]))

1

In [15]:
df1.to_excel("my.xlsx")